# Update some records of Feature Service
The country geometries simplified for gadm level 0 is failing to load on the FE.
This geometries have been simplified previously using mapshaper or ArcGIS Pro.
There is a previous layer used in the NRC that does not fail, we could use that layer for gadm0. The countries failing so far are Russia, USA and Canada (classic, the big countries that cross the antimeridian)

old gadm0 layer (area): https://eowilson.maps.arcgis.com/home/item.html?id=e9b2595e8ee2406b981fd7c00f77df9c
NRC layer: https://eowilson.maps.arcgis.com/home/item.html?id=3c7c5f75cc184a4ca89fe9c8c2154da5
current gadm0 layer (range): https://eowilson.maps.arcgis.com/home/item.html?id=15485a02800a49eba7855b3b79f1d8be

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os


In [2]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [3]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


In [8]:
layers= {'gadm0':'f11bd71ecc4c47aab7735b3cf3e21392',
        'nrc':'3c7c5f75cc184a4ca89fe9c8c2154da5'}

In [9]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

In [10]:
pre= getFlayerFromID(layers['gadm0'])

In [11]:
nrc= getFlayerFromID(layers['nrc'])

In [12]:
print(len(nrc))
nrc.columns

252


Index(['OBJECTID', 'GID_0', 'NAME_0', 'Shape__Area', 'Shape__Length', 'SHAPE'], dtype='object')

In [13]:
print(len(pre))
pre.columns

255


Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'Shape__Area', 'Shape__Length', 'SHAPE'],
      dtype='object')

In [14]:
selp = pre[pre['GID_0'].isin(['RUS','CAN','USA'])]
seln = nrc[nrc['GID_0'].isin(['RUS','CAN','USA'])]

In [ ]:
### To see geoemtries
display(seln.spatial)

In [ ]:
display(selp.spatial)

In [15]:
seln

,OBJECTID,GID_0,NAME_0,Shape__Area,Shape__Length,SHAPE
39,40,CAN,Canada,1694.025240,2210.684826,"{""rings"": [[[-87.035281988421, 65.479153994765..."
248,253,USA,United States,1116.381362,719.210324,"{""rings"": [[[-160.021144980627, -0.39805494820..."
249,400,RUS,Russia,2908.001129,1478.397360,"{""rings"": [[[131.87329129417, 42.9569360103709..."


In [16]:
selp[['GID_0','NAME_0','SHAPE','ObjectId']]

,GID_0,NAME_0,SHAPE,ObjectId
39,CAN,Canada,"{""rings"": [[[-7765692.92845951, 17917763.32315...",40
189,RUS,Russia,"{""rings"": [[[6450032, 16819540], [6449632, 168...",190
234,USA,United States,"{""rings"": [[[-17646464, 12004764], [-17646864,...",235


In [17]:
### Layer to update (gadm0 layer)
item_id = layers['gadm0'] ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('CAN','RUS','USA')")

# ### Layer to retrieve geometries from
new_fset = gis.content.get('3c7c5f75cc184a4ca89fe9c8c2154da5').layers[0].query(where ="GID_0 IN ('CAN','RUS','USA')")
new_fset_features = gis.content.get('3c7c5f75cc184a4ca89fe9c8c2154da5').layers[0].query(where ="GID_0 IN ('CAN','RUS','USA')").features
new_flayer_rows = new_fset.sdf

In [18]:
new_fset.fields

[{'name': 'OBJECTID',
  'type': 'esriFieldTypeOID',
  'alias': 'OBJECTID',
  'sqlType': 'sqlTypeOther',
  'domain': None,
  'defaultValue': None},
 {'name': 'GID_0',
  'type': 'esriFieldTypeString',
  'alias': 'GID_0',
  'sqlType': 'sqlTypeOther',
  'length': 8000,
  'domain': None,
  'defaultValue': None},
 {'name': 'NAME_0',
  'type': 'esriFieldTypeString',
  'alias': 'NAME_0',
  'sqlType': 'sqlTypeOther',
  'length': 8000,
  'domain': None,
  'defaultValue': None},
 {'name': 'Shape__Area',
  'type': 'esriFieldTypeDouble',
  'alias': 'Shape__Area',
  'sqlType': 'sqlTypeDouble',
  'domain': None,
  'defaultValue': None},
 {'name': 'Shape__Length',
  'type': 'esriFieldTypeDouble',
  'alias': 'Shape__Length',
  'sqlType': 'sqlTypeDouble',
  'domain': None,
  'defaultValue': None}]

In [19]:
### https://community.esri.com/t5/arcgis-online-questions/updating-polygon-features-from-a-feature-layer-in/td-p/1043566
from arcgis import geometry
features_to_be_added = []

for index, row in new_flayer_rows.iterrows():
    feature_to_be_updated = deepcopy(fset.features[index])
    input_geometry = new_fset_features[index].geometry
    
    output_geometry = geometry.project(geometries = [input_geometry],
                                       in_sr = new_fset.spatial_reference['wkid'], 
                                       out_sr = fset.spatial_reference['wkid'], 
                                       gis = gis)
    # assign the updated values
    feature_to_be_updated.geometry = output_geometry[0]
    #feature_to_be_updated.attributes['ObjectId'] = int(row['ObjectId'])
    #feature_to_be_updated.attributes['OBJECTID'] = int(row['OBJECTID'])
    
    # #add this to the list of features to be updated
    features_to_be_added.append(feature_to_be_updated)

In [20]:
#fset.features[0]
fset.spatial_reference

{'wkid': 102100, 'latestWkid': 3857}

In [21]:
#features_to_be_added[0].geometry
new_fset.spatial_reference

{'wkid': 4326, 'latestWkid': 4326}

In [22]:
### Add:
#flayer.edit_features(adds = features_to_be_added) 
### Update
flayer.edit_features(updates = features_to_be_added)

{'addResults': [],
 'updateResults': [{'objectId': 40,
   'uniqueId': 40,
   'globalId': None,
   'success': True},
  {'objectId': 190, 'uniqueId': 190, 'globalId': None, 'success': True},
  {'objectId': 235, 'uniqueId': 235, 'globalId': None, 'success': True}],
 'deleteResults': []}

### This added new rows with a copy of the data and new geometries, but it did not update the geometry of the rows.

In [23]:
item_id = 'f11bd71ecc4c47aab7735b3cf3e21392' ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('CAN','RUS','USA')")
df = fset.sdf

In [25]:
df.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'Shape__Area', 'Shape__Length', 'SHAPE'],
      dtype='object')

In [26]:
df[['GID_0','NAME_0','MOL_ID','SHAPE','ObjectId']]

,GID_0,NAME_0,MOL_ID,SHAPE,ObjectId
0,CAN,Canada,40,"{""rings"": [[[-7444521.32800427, 5561807.138174...",40
1,RUS,Russia,190,"{""rings"": [[[-17813472.3754415, -44311.6305776...",190
2,USA,United States,235,"{""rings"": [[[20037507.0671618, 11401317.669004...",235


In [27]:
### Save as shapefile
df.spatial.to_featureclass('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Outputs/copy_CAN_RUS_USA.shp')

'/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Outputs/copy_CAN_RUS_USA.shp'

In [28]:
from arcgis.geometry import SpatialReference
epsg_code = flayer.properties['extent']['spatialReference']['latestWkid']
feature_set = fset
sdf = feature_set.sdf
srs = SpatialReference({"wkid":epsg_code})
sdf.spatial.project(srs)
sdf.spatial.to_featureclass(location='/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Outputs/copy_CAN_RUS_USA.shp')

'/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Outputs/copy_CAN_RUS_USA.shp'

In [29]:
flayer.delete_features(where="objectid > 255")

{'deleteResults': []}

### USA and RUSSIA geometries are switched. Switch them back manually

In [38]:
### Layer to update (gadm0 layer)
item_id = 'f11bd71ecc4c47aab7735b3cf3e21392' ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('RUS','USA')")

In [39]:
df = fset.sdf
df[['GID_0','SHAPE','MOL_ID','ObjectId']]

,GID_0,SHAPE,MOL_ID,ObjectId
0,RUS,"{""rings"": [[[-17813472.3754415, -44311.6305776...",190,190
1,USA,"{""rings"": [[[20037507.0671618, 11401317.669004...",235,235


In [41]:
usa_geom = fset.features[0].geometry
rus_geom = fset.features[1].geometry

In [42]:
features_to_be_added = []

#USA (Data index 1, geometry index 0)
feature_to_be_updated = deepcopy(fset.features[1])
input_geometry = fset.features[0].geometry
feature_to_be_updated.geometry = input_geometry
features_to_be_added.append(feature_to_be_updated)

#Rus (Data index 0, geometry index 1)
feature_to_be_updated = deepcopy(fset.features[0])
input_geometry = fset.features[1].geometry
feature_to_be_updated.geometry = input_geometry
features_to_be_added.append(feature_to_be_updated)

In [43]:
flayer.edit_features(updates = features_to_be_added)

{'addResults': [],
 'updateResults': [{'objectId': 235,
   'uniqueId': 235,
   'globalId': None,
   'success': True},
  {'objectId': 190, 'uniqueId': 190, 'globalId': None, 'success': True}],
 'deleteResults': []}

In [44]:
item_id = 'f11bd71ecc4c47aab7735b3cf3e21392' ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('RUS','USA')")

In [45]:
df1 = fset.sdf
df1[['GID_0','SHAPE','MOL_ID','ObjectId']]

,GID_0,SHAPE,MOL_ID,ObjectId
0,RUS,"{""rings"": [[[20037507.0671618, 11401317.669004...",190,190
1,USA,"{""rings"": [[[-17813472.3754415, -44311.6305776...",235,235
